In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName("Ecomdatapipeline").getOrCreate()

In [0]:
#Reading Files from bronze layer
country_bronze = "/mnt/delta/tables/ecom2/bronze/country"
seller_bronze = "/mnt/delta/tables/ecom2/bronze/sellers"
buyers_bronze = "/mnt/delta/tables/ecom2/bronze/buyers"


In [0]:
buyers_df = spark.read.format("delta").load(buyers_bronze)

In [0]:
#Casting integer columns
integer_columns = [
    'buyers','topbuyers','femalebuyers','malebuyers',
    'topfemalebuyers','topmalebuyers','totalproductsbought',
    'totalproductswished','totalproductsliked','toptotalproductsbought',
    'toptotalproductswished','toptotalproductsliked'
]

for column_name in integer_columns:
    buyers_df = buyers_df.withColumn(column_name,col(column_name).cast(IntegerType()))

In [0]:
#Casting decimal columns

decimal_columns = [
    'topbuyerratio','femalebuyersratio','topfemalebuyersratio',
    'boughtperwishlistratio','boughtperlikeratio','topboughtperwishlistratio',
    'topboughtperlikeratio','meanproductsbought','meanproductswished',
    'meanproductsliked','topmeanproductsbought','topmeanproductswished',
    'topmeanproductsliked','meanofflinedays','topmeanofflinedays',
    'meanfollowers','meanfollowing','topmeanfollowers','topmeanfollowing'
]


for column_name in decimal_columns:
    buyers_df = buyers_df.withColumn(column_name,col(column_name).cast(DecimalType(10,2)))

In [0]:
#Normalize country names
buyers_df = buyers_df.withColumn("country",initcap(col("country")))


for col_name in integer_columns:
    buyers_df = buyers_df.fillna({col_name:0})


#Calculate the ratio of female to male buyers
buyers_df = buyers_df.withColumn("female_to_male_ratio",
                                 round(col("femalebuyers")/(col("malebuyers")+1),2))

#Determine the market potential by comparing wishlist and purchases

buyers_df = buyers_df.withColumn("wishlist_to_purchase_ratio",
                                 round(col("totalproductswished")/(col("totalproductsbought")+1),2))

#Tag countries with a high engagement ratio
high_engagement_threshold = 0.5

buyers_df = buyers_df.withColumn("high_engagement",
                                 when(col("boughtperwishlistratio")>high_engagement_threshold,True)
                                 .otherwise(False))

#Flag markets with increasing female buyer participation
buyers_df  = buyers_df.withColumn("growing_female_market",
                                  when(col("femalebuyersratio")>col("topfemalebuyersratio"),True)
                                  .otherwise(False))


                                  

In [0]:
buyers_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/tables/ecom2/silver/buyers")

In [0]:
sellers_df = spark.read.format("delta").load(seller_bronze)

In [0]:
#Another way to cast column data types

sellers_df = sellers_df\
    .withColumn("nbsellers",col("nbsellers").cast(IntegerType()))\
    .withColumn("meanproductssold",col("meanproductssold").cast(DecimalType(10,2)))\
    .withColumn("meanproductslisted",col("meanproductslisted").cast(DecimalType(10,2)))\
    .withColumn("meansellerpassrate",col("meansellerpassrate").cast(DecimalType(10,2)))\
    .withColumn("totalproductssold",col("totalproductssold").cast(IntegerType()))\
    .withColumn("totalproductslisted",col("totalproductslisted").cast(IntegerType()))\
    .withColumn("meanproductsbought",col("meanproductsbought").cast(DecimalType(10,2)))\
    .withColumn("meanproductswished",col("meanproductswished").cast(DecimalType(10,2)))\
    .withColumn("meanproductsliked",col("meanproductsliked").cast(DecimalType(10,2)))\
    .withColumn("totalbought",col("totalbought").cast(IntegerType()))\
    .withColumn("totalwished",col("totalwished").cast(IntegerType()))\
    .withColumn("totalproductsliked",col("totalproductsliked").cast(IntegerType()))\
    .withColumn("meanfollowers",col("meanfollowers").cast(DecimalType(10,2)))\
    .withColumn("meanfollows",col("meanfollows").cast(DecimalType(10,2)))\
    .withColumn("percentofappusers",col("percentofappusers").cast(DecimalType(10,2)))\
    .withColumn("percentofiosusers",col("percentofiosusers").cast(DecimalType(10,2)))\
    .withColumn("meanseniority",col("meanseniority").cast(DecimalType(10,2)))

In [0]:
#Normalize country names and gender values

sellers_df = sellers_df.withColumn("country",initcap(col("country")))\
                       .withColumn("sex",upper(col("sex")))



#Add a column to categorize the number of sellers

sellers_df = sellers_df.withColumn("seller_size_category",
                                   when(col("nbsellers") < 500,"Small")\
                                   .when((col("nbsellers")>=500) & (col("nbsellers")<2000),"Medium")\
                                   .otherwise("Large"))

# Calculate the mean products listed per seller as an indicator of seller activity
sellers_df = sellers_df.withColumn("mean_products_listed_per_seller",
                                   round(col("totalproductslisted")/col("nbsellers"),2))


# Identify markets with high seller pass rate
sellers_df = sellers_df.withColumn("high_seller_pass_rate",
                                when(col("meansellerpassrate")>0.75,"High")\
                                .otherwise("Normal"))

mean_pass_rate = sellers_df.select(round(avg("meansellerpassrate"),2).alias("avg_pass_rate")).collect()[0]["avg_pass_rate"]

sellers_df = sellers_df.withColumn("meansellerpassrate",
                                   when(col("meansellerpassrate").isNull(),mean_pass_rate)
                                   .otherwise(col("meansellerpassrate")))

In [0]:
sellers_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/tables/ecom2/silver/sellers")

In [0]:
country_df = spark.read.format("delta").load(country_bronze)

In [0]:
country_df = country_df\
    .withColumn("sellers", col("sellers").cast(IntegerType())) \
    .withColumn("topsellers", col("topsellers").cast(IntegerType())) \
    .withColumn("topsellerratio", col("topsellerratio").cast(DecimalType(10, 2))) \
    .withColumn("femalesellersratio", col("femalesellersratio").cast(DecimalType(10, 2))) \
    .withColumn("topfemalesellersratio", col("topfemalesellersratio").cast(DecimalType(10, 2))) \
    .withColumn("femalesellers", col("femalesellers").cast(IntegerType())) \
    .withColumn("malesellers", col("malesellers").cast(IntegerType())) \
    .withColumn("topfemalesellers", col("topfemalesellers").cast(IntegerType())) \
    .withColumn("topmalesellers", col("topmalesellers").cast(IntegerType())) \
    .withColumn("countrysoldratio", col("countrysoldratio").cast(DecimalType(10, 2))) \
    .withColumn("bestsoldratio", col("bestsoldratio").cast(DecimalType(10, 2))) \
    .withColumn("toptotalproductssold", col("toptotalproductssold").cast(IntegerType())) \
    .withColumn("totalproductssold", col("totalproductssold").cast(IntegerType())) \
    .withColumn("toptotalproductslisted", col("toptotalproductslisted").cast(IntegerType())) \
    .withColumn("totalproductslisted", col("totalproductslisted").cast(IntegerType())) \
    .withColumn("topmeanproductssold", col("topmeanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("topmeanproductslisted", col("topmeanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meanproductssold", col("meanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("meanproductslisted", col("meanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meanofflinedays", col("meanofflinedays").cast(DecimalType(10, 2))) \
    .withColumn("topmeanofflinedays", col("topmeanofflinedays").cast(DecimalType(10, 2))) \
    .withColumn("meanfollowers", col("meanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("meanfollowing", col("meanfollowing").cast(DecimalType(10, 2))) \
    .withColumn("topmeanfollowers", col("topmeanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("topmeanfollowing", col("topmeanfollowing").cast(DecimalType(10, 2)))

        

In [0]:
#Calculate the ratio of top sellers to total sellers

country_df = country_df.withColumn("top_seller_ratio",
                                   round(col("topsellers")/col("sellers"),2))


# Countries with a high ratio of female sellers
country_df = country_df.withColumn("high_female_seller_ratio",
                                   when(col("femalesellersratio")>0.5,True).otherwise(False))


# Adding a performance indicator based on the sold/listed ratio
country_df = country_df.withColumn("performance_indicator",
                                   round(col("toptotalproductssold")/(col("toptotalproductslisted")+1),2))

#Flag countries with exceptionally high performance
performance_threshold = 0.8

country_df = country_df.withColumn("high_performance",
                                   when(col("performance_indicator")>performance_threshold,True).otherwise(False))


country_df = country_df.withColumn("activity_level",
                                   when(col("meanofflinedays")<30,"Highly active")
                                   .when((col("meanofflinedays")>=30) & (col("meanofflinedays")<60),"Moderately Active")
                                   .otherwise("Low Activity"))



In [0]:
country_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/tables/ecom2/silver/country")